# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import pandas as pd
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Input,GlobalAveragePooling1D
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
import joblib
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [2]:
excel_path = 'data/database/dataBase.xlsx'  
df = pd.read_excel(excel_path)

file_paths = df['path'].tolist()
id = df['id'].tolist()
labels = df['person'].tolist()

n = len(file_paths)  
sample_size = n // 25

file_paths_sample = file_paths[:sample_size]
id_sample = id[:sample_size]
labels_sample = labels[:sample_size]
id_sample_unknown = id[sample_size + 27: 1170]
unknown_file_paths = file_paths[sample_size + 27 :1170]
unknown_labels = ["unknown"] * len(unknown_file_paths)

print(len(file_paths_sample))  
print(len(id_sample))
print(len(labels_sample))
print(len(unknown_file_paths))
print(len(unknown_labels))

1059
1059
1059
84
84


In [3]:
unique_labels = list(set(labels_sample))
print(unique_labels)
len(unique_labels)
print(labels_sample)
len(unique_labels)

['TVYM', 'TPBL', 'XRJO', 'NUID', 'UQEN', 'DMLN', 'IRVN', 'IQMH', 'IZDT', 'HBXT', 'YQXI', 'EMCL', 'UXOD']
['UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD', 'UXOD'

13

In [4]:
paths = []

paths.append("data/test/3 (1).wav")  
for i in range(4, 92):  
    paths.append(f"data/test/{i}.wav")

print(paths)

paths_unknown_human_voices = []

for i in range(1, 101):  
    paths_unknown_human_voices.append(f"unsounds/human_voice_{i}.wav")

print(paths_unknown_human_voices)


['data/test/3 (1).wav', 'data/test/4.wav', 'data/test/5.wav', 'data/test/6.wav', 'data/test/7.wav', 'data/test/8.wav', 'data/test/9.wav', 'data/test/10.wav', 'data/test/11.wav', 'data/test/12.wav', 'data/test/13.wav', 'data/test/14.wav', 'data/test/15.wav', 'data/test/16.wav', 'data/test/17.wav', 'data/test/18.wav', 'data/test/19.wav', 'data/test/20.wav', 'data/test/21.wav', 'data/test/22.wav', 'data/test/23.wav', 'data/test/24.wav', 'data/test/25.wav', 'data/test/26.wav', 'data/test/27.wav', 'data/test/28.wav', 'data/test/29.wav', 'data/test/30.wav', 'data/test/31.wav', 'data/test/32.wav', 'data/test/33.wav', 'data/test/34.wav', 'data/test/35.wav', 'data/test/36.wav', 'data/test/37.wav', 'data/test/38.wav', 'data/test/39.wav', 'data/test/40.wav', 'data/test/41.wav', 'data/test/42.wav', 'data/test/43.wav', 'data/test/44.wav', 'data/test/45.wav', 'data/test/46.wav', 'data/test/47.wav', 'data/test/48.wav', 'data/test/49.wav', 'data/test/50.wav', 'data/test/51.wav', 'data/test/52.wav', 'd

In [5]:
label_ali = "ali"
labels_ali = [label_ali] * len(paths)

unknown_labels = ["unknown"] * (len(unknown_file_paths) + len(paths_unknown_human_voices))
labels_all = labels_sample + labels_ali + unknown_labels

# unwanted_keywords = ["XRJO"]

audio_paths_new_unknown = [f"{path}{str(identifier)}.wav" for path, identifier in zip(unknown_file_paths, id_sample_unknown)]

audio_paths_new = [f"{path}{str(identifier)}.wav" for path, identifier in zip(file_paths_sample, id_sample)]
# filtered_audio_paths_unknown = [path for path in audio_paths_new_unknown if not any(keyword in path for keyword in unwanted_keywords)]
audio_paths_all = audio_paths_new + paths + audio_paths_new_unknown + paths_unknown_human_voices

audio_paths_np = np.array(audio_paths_all).reshape(-1, 1)

ros = RandomOverSampler(random_state=42)
X_balanced, y_balanced = ros.fit_resample(audio_paths_np, labels_all)

print(Counter(y_balanced))
print(audio_paths_all)

# UXOD HBXT NUID IQMH IRVN TVYM YQXI TPBL EMCL DMLN UQEN IZDT XRJO

Counter({'UXOD': 184, 'HBXT': 184, 'NUID': 184, 'IQMH': 184, 'IRVN': 184, 'TVYM': 184, 'YQXI': 184, 'TPBL': 184, 'EMCL': 184, 'DMLN': 184, 'UQEN': 184, 'IZDT': 184, 'XRJO': 184, 'ali': 184, 'unknown': 184})
['data/database/ac/ac_UXOD_BTMEAPU.wav', 'data/database/ac/ac_UXOD_YJTEPUH.wav', 'data/database/ac/ac_UXOD_CKTQBAP.wav', 'data/database/ac/ac_UXOD_YCJKIMB.wav', 'data/database/ac/ac_UXOD_YJKRCPO.wav', 'data/database/ac/ac_UXOD_HJKNTRU.wav', 'data/database/ac/ac_UXOD_XYGONEW.wav', 'data/database/ac/ac_UXOD_HRWYZVK.wav', 'data/database/ac/ac_UXOD_AQNLZBD.wav', 'data/database/asagi/asagi_UXOD_OQFUBAC.wav', 'data/database/asagi/asagi_UXOD_GSYOHNZ.wav', 'data/database/asagi/asagi_UXOD_QIUOVND.wav', 'data/database/asagi/asagi_UXOD_NZSJFUC.wav', 'data/database/asagi/asagi_UXOD_ZPKXAQE.wav', 'data/database/asagi/asagi_UXOD_CLDMVRG.wav', 'data/database/asagi/asagi_UXOD_GLZBNDK.wav', 'data/database/baslat/baslat_UXOD_IRSGZHU.wav', 'data/database/baslat/baslat_UXOD_VBLCNGQ.wav', 'data/database

In [6]:
def add_noise(audio, noise_level=0.005):
    noise = np.random.randn(len(audio)) * noise_level
    return audio + noise

# def time_stretch(audio):
#     return librosa.effects.time_stretch(audio, rate=0.9)
# 
# def pitch_shift(audio, sample_rate, n_steps=2):
#     return librosa.effects.pitch_shift(audio, sr=sample_rate, n_steps=n_steps)
# 
# def time_shift(audio, shift_max=0.1):
#     shift = np.random.randint(int(-shift_max * len(audio)), int(shift_max * len(audio)))
#     return np.roll(audio, shift)

features_for_anomaly = []
def extract_features(file_path, sample_rate=16000, augment=False):
    try:
        if isinstance(file_path, np.ndarray):
            file_path = file_path.item()
            
        audio, _ = librosa.load(file_path, sr=sample_rate)
        augmented_audio = [audio]  

        if augment:
            augmented_audio.append(add_noise(audio))  
            # augmented_audio.append(time_stretch(audio))
            # augmented_audio.append(time_shift(audio))
            # augmented_audio.append(pitch_shift(audio, sample_rate, n_steps=np.random.randint(-3, 3)))    

        all_features = []

        for aug_audio in augmented_audio:
            mfcc = librosa.feature.mfcc(y=aug_audio, sr=sample_rate, n_mfcc=45)

            spectral_centroid = librosa.feature.spectral_centroid(y=aug_audio, sr=sample_rate)

            zcr = librosa.feature.zero_crossing_rate(y=aug_audio)
            
            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max) 

            features = np.hstack([
                np.mean(mfcc.T, axis=0),
                np.mean(spectral_centroid),
                np.mean(zcr),                       
                np.mean(mel_spec_db)
            ])
            all_features.append(features)
            features_for_anomaly.append(features)

        return all_features

    except Exception as e:
        print(f"Özellik çıkarımı sırasında hata oluştu: {e}")
        return None

if isinstance(X_balanced, np.ndarray):
    X_balanced = X_balanced.tolist()

if isinstance(y_balanced, np.ndarray):  
    y_balanced = y_balanced.tolist()

X_features = []
y_final = []

for idx, path in enumerate(X_balanced):
    features_list = extract_features(path[0], augment=True)

    if features_list is not None:
        for features in features_list:
            X_features.append(features)
            y_final.append(y_balanced[idx])  

X_features = np.array(X_features)
y_final = np.array(y_final)


In [7]:
iso_forest = IsolationForest(contamination="auto", random_state=42)
iso_forest.fit(np.array(features_for_anomaly))

test_file = "audio.wav"
test_features = np.array(extract_features(test_file)).reshape(1, -1)

score = iso_forest.decision_function(test_features)
prediction = iso_forest.predict(test_features)

if prediction[0] == -1:
    print(f"{test_file} anormal bir ses içeriyor!")
else:
    print(f"{test_file} normal bir ses.")


audio.wav normal bir ses.


In [8]:
joblib.dump(iso_forest, "anomaly_agent/anomaly_agent.pkl")
print("Model başarıyla kaydedildi!")

Model başarıyla kaydedildi!


In [9]:
def create_model(input_shape, output_shape):    
    model = Sequential([
        Input(shape=(input_shape, 1)),

        Conv1D(64, kernel_size=5, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(128, kernel_size=5, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(256, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(512, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.4),

        GlobalAveragePooling1D(),  
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(output_shape, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(X, y, k=4, 
                      model_save_path="models/modelV14/model_mk3.keras",
                      encoder_save_path="models/modelV14/label_encoder.pkl",
                      scaler_save_path="models/modelV14/scaler.pkl"):

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []
    best_accuracy = 0
    best_model = None
    best_label_encoder = None
    best_scaler = None
    best_y_test = None
    best_y_pred = None
    best_y_true_labels = None

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {fold + 1}/{k}")

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        label_encoder = LabelEncoder()
        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        y_train_encoded = to_categorical(label_encoder.fit_transform(y_train))
        y_test_encoded = to_categorical(label_encoder.transform(y_test))

        model = create_model(X_train.shape[1], len(label_encoder.classes_))

        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6, verbose=1)

        model.fit(
            X_train, y_train_encoded,
            validation_split=0.2,
            epochs=50,
            batch_size=32,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
        print(f"Fold {fold + 1} Accuracy: {test_accuracy:.2f}")
        fold_accuracies.append(test_accuracy)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model = model
            best_label_encoder = label_encoder
            best_scaler = scaler
            best_y_test = y_test
            best_y_true_labels = label_encoder.transform(y_test)
            best_y_pred = model.predict(X_test[..., np.newaxis])

    print(f"Average K-Fold Accuracy: {np.mean(fold_accuracies):.2f}")

    if best_model:
        best_model.save(model_save_path)
        print(f"En iyi model kaydedildi: {model_save_path}")

        joblib.dump(best_label_encoder, encoder_save_path)
        print(f"En iyi LabelEncoder kaydedildi: {encoder_save_path}")

        joblib.dump(best_scaler, scaler_save_path)
        print(f"En iyi Scaler kaydedildi: {scaler_save_path}")
        
        y_pred_labels = np.argmax(best_y_pred, axis=1)
        cm = confusion_matrix(best_y_true_labels, y_pred_labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_label_encoder.classes_)

        plt.figure(figsize=(10, 8))
        disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
        plt.title("Confusion Matrix - En İyi Fold")
        plt.tight_layout()
        plt.savefig("confusion_matrix.png", dpi=300)
        plt.close()
        print("Confusion matrix PNG olarak kaydedildi: confusion_matrix.png")


In [10]:
train_model(X_features, y_final)

Fold 1/4
AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.2723 - loss: 2.4923 - val_accuracy: 0.0580 - val_loss: 3.9553 - learning_rate: 0.0010
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7020 - loss: 0.9827 - val_accuracy: 0.0423 - val_loss: 5.0745 - learning_rate: 0.0010
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8196 - loss: 0.5680 - val_accuracy: 0.2476 - val_loss: 4.6697 - learning_rate: 0.0010
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8758 - loss: 0.3874 - val_accuracy: 0.5531 - val_loss: 1.5828 - learning_rate: 0.0010
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8888 - loss: 0.3282 - val_accuracy: 0.8527 - val_loss: 0.3713 - learning_rate: 0.0010
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.9135 - loss: 0.2437 - val_accuracy: 0.9336 - val_loss: 0.1655 - learning_rate: 0.0010
Epoch 7

<Figure size 1000x800 with 0 Axes>

In [11]:
def predict_audio(file_path, 
                  model_path="models/modelV14/model_mk3.keras", 
                  encoder_path="models/modelV14/label_encoder.pkl", 
                  scaler_path="models/modelV14/scaler.pkl", 
                  threshold=0.95):

    model = load_model(model_path)
    label_encoder = joblib.load(encoder_path)
    scaler = joblib.load(scaler_path)

    def extract_features(file_path, sample_rate=16000):
        try:
            audio, _ = librosa.load(file_path, sr=sample_rate)    
            # **MFCC (40 boyut)**
            mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=45)
            mfcc_mean = np.mean(mfcc.T, axis=0)  
    
            # **Spektral Centroid (1 boyut)**
            spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
            spectral_centroid_mean = np.mean(spectral_centroid)
    
            # **Zero-Crossing Rate (1 boyut)**
            zcr = librosa.feature.zero_crossing_rate(audio)
            zcr_mean = np.mean(zcr)
            
            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max) 
    
            features = np.hstack([mfcc_mean, spectral_centroid_mean, zcr_mean, np.mean(mel_spec_db)])
    
            return features
    
        except Exception as e:
            print(f"⚠️ Hata oluştu: {e}")
            return None

    features = extract_features(file_path)
    if features is None:
        return "Özellik çıkarımı başarısız oldu."
    
    features = features.reshape(1, -1)  
    features = scaler.transform(features)  


    predictions = model.predict(features)
    predicted_class = np.argmax(predictions, axis=1)  
    predicted_prob = np.max(predictions)  

    if predicted_prob < threshold:
        return "unknown"
    
    label = label_encoder.inverse_transform(predicted_class)
    return label[0]

In [21]:
# UXOD HBXT NUID IQMH IRVN TVYM YQXI TPBL EMCL DMLN UQEN IZDT XRJO
file_path = "Kayıt+(54).wav"  
predicted_label = predict_audio(file_path)
print(f"Tahmin edilen etiket: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Tahmin edilen etiket: ali


In [48]:
from scipy.io.wavfile import write
def generate_human_like_voice(filename="human_voice.wav", duration=7, sample_rate=16000, frequency=150):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    
    # Temel dalga + Harmonikler
    human_voice = 0.5 * np.sin(2 * np.pi * frequency * t)
    human_voice += 0.25 * np.sin(2 * np.pi * frequency * 2 * t)
    human_voice += 0.1 * np.sin(2 * np.pi * frequency * 3 * t)
    
    # Frekans modülasyonu (FM)
    human_voice = 0.5 * np.sin(2 * np.pi * frequency * t + 0.1 * np.sin(2 * np.pi * 0.1 * t))
    
    # Amplitüd modülasyonu (AM)
    human_voice *= (1 + 0.1 * np.sin(2 * np.pi * 0.05 * t))
    
    # Ses genliğini normalize etme
    human_voice = np.int16(human_voice / np.max(np.abs(human_voice)) * 32767)
    
    # WAV dosyasına yazma
    write(filename, sample_rate, human_voice)
    print(f"{filename} dosyası oluşturuldu.")

for i in range(100):
    frequency_variation = 150 + np.random.randint(-50, 50)  # Rastgele frekans değişimi
    filename = f"unsounds/human_voice_{i+1}.wav"
    generate_human_like_voice(filename=filename, frequency=frequency_variation)

unsounds/human_voice_1.wav dosyası oluşturuldu.
unsounds/human_voice_2.wav dosyası oluşturuldu.
unsounds/human_voice_3.wav dosyası oluşturuldu.
unsounds/human_voice_4.wav dosyası oluşturuldu.
unsounds/human_voice_5.wav dosyası oluşturuldu.
unsounds/human_voice_6.wav dosyası oluşturuldu.
unsounds/human_voice_7.wav dosyası oluşturuldu.
unsounds/human_voice_8.wav dosyası oluşturuldu.
unsounds/human_voice_9.wav dosyası oluşturuldu.
unsounds/human_voice_10.wav dosyası oluşturuldu.
unsounds/human_voice_11.wav dosyası oluşturuldu.
unsounds/human_voice_12.wav dosyası oluşturuldu.
unsounds/human_voice_13.wav dosyası oluşturuldu.
unsounds/human_voice_14.wav dosyası oluşturuldu.
unsounds/human_voice_15.wav dosyası oluşturuldu.
unsounds/human_voice_16.wav dosyası oluşturuldu.
unsounds/human_voice_17.wav dosyası oluşturuldu.
unsounds/human_voice_18.wav dosyası oluşturuldu.
unsounds/human_voice_19.wav dosyası oluşturuldu.
unsounds/human_voice_20.wav dosyası oluşturuldu.
unsounds/human_voice_21.wav d